# Load libraries

In [2]:
import pandas as pd
import numpy as np
import spacy
from collections import Counter
import json
import pyodbc 
import re
from spellchecker import SpellChecker
import hunspell
import wordninja
from utils import clean_text

hyperparams = False

#!python -m spacy download es_core_news_md
nlp = spacy.load('es_core_news_md')
nlp.pipe_names

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

## Data cleaning

In [2]:
# connection parameters
server = '52.224.6.206' 
database = 'Previsora_SECOP' 
username = 'admin_usersql' 
password = 'admin_usersql'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
#cursor = cnxn.cursor()

#Sample select query
query = "SELECT * FROM dbo.EntrenamientoSECOP"
df = pd.read_sql(query,cnxn)

# Erase rows with null values
df = df.loc[~df['ObjetoProceso'].isnull()]

# Erase rows where text length is less than 20
df = df.loc[df['ObjetoProceso'].apply(len) > 20]

# Separate label column and make lower case all text
text = df['ObjetoProceso']
#text = np.array([desc.lower() for desc in text])
len(text)

6081

In [ ]:
# df1 = pd.read_excel('data/Revision_de_Datos_SECOP_I_y_II_Clean.xlsx', 'SECOP I')
# df1 = df1.loc[~df1['Detalle del Objeto a Contratar'].isnull()]

# df2 = pd.read_excel('data/Revision_de_Datos_SECOP_I_y_II_Clean.xlsx', 'SECOP II')
# df2 = df2.loc[~df2['Detalle del Objeto a Contratar'].isnull()]

# df = df1.append(df2, ignore_index=True)
# df = df.loc[df['Detalle del Objeto a Contratar'].apply(len) > 20] # Definir con el negocio este umbral
# df.info()

In [7]:
# df['text length'] = df['Detalle del Objeto a Contratar'].apply(len)
# df.sort_values('text length')

,Detalle del Objeto a Contratar,Calificación,text length
2071,renovacion de polizas,1,21
1724,ADQUISICION DE POLIZA,1,21
587,ADQUISICION DE POLIZA,1,21
278,PAGO POLIZAS DE MANEJO,1,22
2363,Adquisición Póliza IRF,1,22
...,...,...,...
1409,LA UNIVERSIDAD NACIONAL ABIERTA Y A DISTANCIA ...,1,982
2022,EL PRESENTE CONTRATO TIENE COMO OBJETO QUE EL ...,1,987
2021,EL PRESENTE CONTRATO TIENE COMO OBJETO QUE EL ...,1,987
2023,EL PRESENTE CONTRATO TIENE COMO OBJETO QUE EL ...,1,987


# Misspelled words correction

## -Custom dictionary construction

In [5]:
spell = SpellChecker(language='es')
spell.known(['adquisicion', 'renovacion', 'institucion'])
spell.correction('institucion')

'institución'

In [6]:
spanish_dict = hunspell.HunSpell('dict/es_CO.dic', 'dict/es_CO.aff')
spanish_dict.spell('institución')

True

In [8]:
spell = SpellChecker(language='es')
spanish_dict = hunspell.HunSpell('dict/es_CO.dic', 'dict/es_CO.aff')

# Add Colombian municipalities
municipios = np.loadtxt('dict/municipios_colombia.csv', dtype=str).tolist()
for municipio in municipios:
    spanish_dict.add(municipio)

# Add Colombian departments
departments = ['amazonas', 'antioquia', 'arauca', 'atlantico', 'bogotá', 'bolívar', 'boyacá', 'caldas', 'caquetá', 'casanare',
            'cauca', 'cesar', 'chocó', 'córdoba', 'cundinamarca', 'guainía', 'guaviare', 'huila', 'guajira', 'magdalena', 
            'meta', 'nariño', 'putumayo', 'quindío', 'risaralda', 'providencia', 
            'santander', 'sucre', 'tolima', 'valle', 'vaupés', 'vichada']
for departmet in departments:
    spanish_dict.add(departmet) 

corregimientos = ['usaquen', 'chapinero', 'fusagasugá', 'tunjuelito', 'engativá', 'teusaquillo', 'usme', 'carlosama']
for corregimiento in corregimientos:
    spanish_dict.add(corregimiento) 

# Add context words (improove with lemmalization)
context_words = ['soat', 'extracontractual', 'interadministrativo', 'multirriesgo', 'contraloría', 'interadministrativos',
                'comodato', 'pyme', 'presupuestal', 'volqueta', 'invias', 'ltda', 'multirriesgos', 'extrapatrimoniales',
                'transmilenio', 'arl', 'inpec', 'volquetas', 'buseta', 'invima', 'parafiscales', 'ideam', 'sisbén', 
                'colpensiones', 'icfes']
for word in context_words:
    spanish_dict.add(word)

# Correct initiality some words
text_corrected = []
for x in text:
    # Remove puntuation before use spellchecker
    x = x.lower()
    x = clean_text(x)
    misspelled = list(spell.unknown(x.split(' ')))
    for word in misspelled:
        if word[-3:] == 'ion':            
            x = x.replace(word, spell.correction(word))
    text_corrected.append(x)

len(text_corrected)

6081

In [10]:
# Create word frequency dictionary
known_list = []
for x in text_corrected:
    known = [word for word in x.split(' ') if spanish_dict.spell(word)]
    known_list += known
known_list = list(filter(lambda x: x != '', known_list))

word_frequency = dict(Counter(known_list))
# print(len(word_frequency)

# Remove words with one or two characters and just one occurrency
# words2remove = {key for key, freq in word_frequency.items() if ((freq == 1) & (len(key) < 3))} 
# for word in words2remove:
#     print(word)
#     word_frequency.pop(word)    

#print(sorted(word_frequency.items(), key=lambda x: x[1], reverse=False))

with open('dict/custom_dict.json', 'w') as fp:
    json.dump(word_frequency, fp, sort_keys=True, indent=4)

544


## Split concatenated words

In [ ]:
f = open('dict/custom_dict.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
data = sorted(data.items(), key=lambda x: x[1], reverse=True)
words = list(dict(data).keys()) 
np.savetxt('dict/my_lang.txt.gz', words, fmt='%s')

In [24]:
lm = wordninja.LanguageModel('dict/my_lang.txt.gz')
lm.split('porlos')

['por', 'los']

## Sample code to use the word corrector

In [11]:

spell = SpellChecker(language=None, case_sensitive=False, distance=2)
spell.word_frequency.load_dictionary('dict/custom_dict.json') 
spell.correction('actualizacion')

'actualización'